## The Ground Modeling Challenge for AgriPV Application

<img src="MicrosoftTeams-image.png">

### Solution Steps ###
- Setup of variables
- Generating the scenes
- Mapping the ground irradiance

## 1. Load Bifacial Radiance and other essential packages

In [1]:
import bifacial_radiance
import numpy as np
import os # this operative system to do teh relative-path testfolder for this example.
import pprint    # We will be pretty-printing the trackerdictionary throughout to show its structure.
from pathlib import Path

## 2. Define all the system variables

In [ ]:
testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP')


timestamp = 4020 # Noon, June 17th.
simulationName = 'GroundChallenge'    # Optionally adding a simulation name when defning RadianceObj

# Surface    
#albedo = " green grass", which is not one of the default choices in the material list

#Location
lat = 40.1217  # Given for the project site at Colorado
lon = -105.1310  # Given for the project site at Colorado

# MakeModule Parameters

moduletype='PrismSolar'
#numpanels = 1  # This site have 1 module in Y-direction
x = 1  
y = 2
#xgap = 0.15 # Leaving 15 centimeters between modules on x direction
#ygap = 0.10 # Leaving 10 centimeters between modules on y direction
zgap = 0 # no gap to torquetube.
sensorsy = 6  # this will give 6 sensors per module in y-direction
sensorsx = 3   # this will give 3 sensors per module in x-direction

torquetube = True
axisofrotationTorqueTube = True 
diameter = 0.15  # 15 cm diameter for the torquetube
tubetype = 'square'    # Put the right keyword upon reading the document
material = 'black'   # Torque tube of this material (0% reflectivity)

# Scene variables
nMods = 30
nRows = 7
hub_height = 1.8 # meters
pitch = 5.1816 # meters      # Pitch is the known parameter 
albedo = 0.2  #'Grass'     # ground albedo

#azimuth_ang=180 # Facing south 
#axis_azimuth should have a default value of 180
#tilt = 40 # tilt is not given; lat of the site taken as the tilt

# for ground irradiation simulation should we consider 2*pitch+y-length of panel?

# Traking parameters
cumulativesky = False
limit_angle = 60 # tracker rotation limit angle
angledelta = 0.01 # we will be doing hourly simulation, we want the angle to be as close to real tracking as possible.
backtrack = True 

## 3. Create Radiance Object including Albedo and Weather

In [ ]:
demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) 
epwfile = demo.getEPW(lat, lon) 
metdata = demo.readEPW(epwfile)

## 4. Make Module

In [ ]:
moduleDict = demo.makeModule(name=moduletype, torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=material, 
                zgap=zgap, axisofrotationTorqueTube=axisofrotationTorqueTube)

#sceneDict = {'tilt':tilt,'pitch': pitch,'hub_height':hub_height,'module_type':moduletype, 'nMods': nMods, 'nRows': nRows}  
#scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
#octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object fil|es into a .oct file.


At this point we could use the gendaylit() function for the single timestamp and generate the oct file as below:

In [ ]:
# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) 
epwfile = demo.getEPW(lat, lon) 
metdata = demo.readEPW(epwfile)
demo.gendaylit(timestamp)  # We are simulating only one hour at a time. 

sceneDict = {'pitch': pitch,'hub_height':hub_height,'module_type':moduletype, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 30 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object fil|es into a .oct file.

## 5. Calculate GCR

In [ ]:
cw = 1  # Collector Width, CW = 1 as given
gcr = cw/pitch
print("GCR:",gcr)

## 6. Generate the Sky for the Tracking Angles

In [ ]:
startdate = '06/17'     
enddate = '06/17' #In this case, we are looking to generate tracking scenarios for one day as opposed to a single hour
trackerdict = demo.gendaylit1axis(startdate=startdate, enddate=enddate)
# By the documentation, in gendaylit workflow the set1axis() function would also return a trackerdict


In [ ]:
#checking our trackdict

print ("\nTrimmed trackerdict by gendaylit1axis to start and enddate length: %s " % (len(trackerdict)))
print ("")
trackerkeys = sorted(trackerdict.keys())
print ("Option of hours are: ", trackerkeys)
print ("")
print ("Contents of trackerdict for sample hour:")
pprint.pprint(trackerdict[trackerkeys[5]])

## Make Scene1 Axis

In [ ]:
# making the different scenes for the 1-axis tracking for the dates in trackerdict2.

sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows}  

## Make the 1-axis Tracking Scene

In [ ]:
#demo.set1axis(limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, cumulativesky = cumulativesky)
demo.gendaylit1axis(startdate=startdate, enddate=enddate)
demo.makeScene1axis(moduletype=moduletype,sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
demo.makeOct1axis()
demo.analysis1axis()

Analysis of Ground Irradiance

In [ ]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
sensorsy = 6
frontscan, backscan = analysis.moduleAnalysis(scene, sensorsy=sensorsy)

In [ ]:
groundscan = frontscan

In [ ]:
# This is where we are going to tweak

groundscan['zstart'] = 0  # setting it 0 cm from the ground since we are simulating for ground surface irradiation
groundscan['zinc'] = 0   # no tilt necessary. 
groundscan['yinc'] = pitch/(sensorsy-1)   # no tilt necessary. 
groundscan

In [ ]:
analysis.analysis(octfile, simulationname+"_groundscan", groundscan, backscan)  # compare the back vs front irradiance
#Question: Should it be analysis1axis() instead?

In [ ]:
analysis.Wm2Front

In [ ]:
demo.makeOct1axis(singleindex='06_17_12')
results = demo.analysis1axis(singleindex='06_17_12')
print('\n\nHourly bifi gain: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))

### Code-looping for all hours of the day

In [ ]:
# If we are showing the result for the single hour
print ("\n Contents of trackerdict for sample hour after analysis1axis: ")
pprint.pprint(trackerdict[trackerkeys[5]])

# If we are showing the result for the single hour in better detail
pprint.pprint(trackerdict[trackerkeys[5]]['AnalysisObj'].__dict__)

# If we are looping for all the hours

for key in trackerdict.keys():
    demo.makeOct1axis(singleindex=key)
    results=demo.analysis1axis(singleindex=key)

print('Accumulated hourly bifi gain for the day: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))